In [1]:
import re, json, pathlib, logging, time, argparse, pprint, warnings
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
from natsort import index_natsorted
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import polmap.clean_docs as cldc
import postprocess.postprocess as pspr
from docx2python import docx2python
import docx2python.iterators as d2piter


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [2]:
indicators=pd.read_excel('GIF_2021_refinement_English.xlsx')
indicators.fillna('', inplace=True)
indicators.head(50)

/home/ermaro/miniconda3/envs/polmap/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Targets,Indicators,UNSD
0,Goal 1. End poverty in all its forms everywhere,,
1,"1.1 By 2030, eradicate extreme poverty for all...",1.1.1 Proportion of the population living belo...,C010101
2,"1.2 By 2030, reduce at least by half the propo...",1.2.1 Proportion of population living below th...,C010201
3,,"1.2.2 Proportion of men, women and children of...",C010202
4,1.3 Implement nationally appropriate social pr...,1.3.1 Proportion of population covered by soci...,C010301
5,"1.4 By 2030, ensure that all men and women, in...",1.4.1 Proportion of population living in house...,C010401
6,,1.4.2 Proportion of total adult population wit...,C010402
7,"1.5 By 2030, build the resilience of the poor ...","1.5.1 Number of deaths, missing persons and di...",C200303
8,,1.5.2 Direct economic loss attributed to disas...,C010502
9,,1.5.3 Number of countries that adopt and imple...,C200304


In [5]:
map_df=pd.DataFrame(columns=['Goal','Target', 'Indicator'])
for row in indicators.itertuples():

    goal = re.match(r'(Goal \d{,2}\.)',row.Targets).group(0) if re.match(r'(Goal \d{,2}\.)',row.Targets) else ''
    target = re.match(r'(\d{,2}\.\w{,2})',row.Targets).group(0) if re.match(r'(\d{,2}\.\w{,2})',row.Targets) else ''
    indicator = re.match(r'(\d{,2}\.\w{,2}\.\w{,2})',row.Indicators).group(0) if re.match(r'(\d{,2}\.\w{,2}\.\w{,2})',row.Indicators) else ''
    
    if goal:
        lastgoal=goal
    if not goal:
        goal = lastgoal

    if not (target or indicator):
        target = f"{goal.replace('Goal', 'Target')}0"
    elif target:
        target = f'Target {target}'
    elif goal and indicator:
        target = f"Target {indicator.split('.')[0]}.{indicator.split('.')[1]}"

    goal = goal.replace('Goal', 'SDG')

    new_row = {'Goal':goal.replace('.', ''), 'Target': target, 'Indicator': indicator}
    map_df = map_df.append(new_row,ignore_index=True)


In [6]:
# map_df = map_df.groupby(['Target'])['Indicator'].apply(', '.join)
# map_df = map_df.sort_values(key=lambda x: np.argsort(index_natsorted(map_df["Goal"])))

In [ ]:
map_df.head(50)

In [ ]:
map_df.to_excel('indicators.xlsx')

In [ ]:
def makeindicatorlist(total_summary):
    # get target with count > 0
    # do a left join with indicators.xlsx

    # loop over the dataframe to buil the json file for indicators

    # use chardect and bs4 to implement encoding detection in polmap/